# Modeling convolved task-induced activity in a (coarser?) neural network accounting for network connectivity


Set up environment

Helper functions

## BOLD level


### Resting state


#### Convolve resting state activity

In [ ]:
TRLength = 100

restfmri = model.convolveTimeseries(restdata,
                                    samplingrate=dt,
                                    TRLength=TRLength)

In [ ]:
plt.figure()
plt.imshow(restfmri, aspect='auto',origin='lower')
plt.title('Simulated fMRI resting-state activity', y=1.04, fontsize=16)
plt.ylabel('Regions',fontsize=14)
plt.xlabel('Time (s)',fontsize=14)
plt.colorbar()

#### Recover connectivity matrix from resting state activity

In [ ]:
fcmat = np.corrcoef(restfmri)
# 0 out the diagonal
np.fill_diagonal(fcmat,0)

plt.figure()
plt.imshow(fcmat, origin='lower')
plt.colorbar()
plt.title('Simulated fMRI resting-state FC \nSingle Subject\nPearsonFC', y=1.04, fontsize=16)
plt.xlabel('Regions',fontsize=14)
plt.ylabel('Regions', fontsize=14)
plt.tight_layout()

In [ ]:
fcmat_mreg = mreg.multregressionconnectivity(restfmri)
# 0 out the diagonal
np.fill_diagonal(fcmat_mreg,0)

plt.figure()
plt.imshow(fcmat_mreg, origin='lower')
plt.colorbar()
plt.title('Simulated fMRI resting-state FC\nSingle Subject\nMultipleRegressionFC', y=1.04, fontsize=16)
plt.xlabel('Regions',fontsize=14)
plt.ylabel('Regions', fontsize=14)
plt.tight_layout()

How **quantitatively** similar is the fc_mat to the weight matrix G?

In [ ]:
plt.scatter(fcmat.flatten(), G.flatten())
np.corrcoef(fcmat.flatten(), G.flatten())

In [ ]:
plt.scatter(fcmat_mreg.flatten(), G.flatten())
np.corrcoef(fcmat_mreg.flatten(), G.flatten())

### Task fmri

##### Convolve task activity 


In [ ]:
samplingrate=1.0
# Convolve simulated task data into fMRI signal
task_bold = model.convolveTimeseries(taskdata[task],
                               samplingrate=samplingrate, 
                               TRLength=TRLength)


In [ ]:
plt.figure()
plt.imshow(task_bold, aspect='auto',origin='lower')
plt.title('Simulated single topdown task fMRI activity', y=1.04, fontsize=16)
plt.ylabel('Regions',fontsize=14)
plt.xlabel('Time (s)',fontsize=14)
plt.colorbar()

##### Run GLM on convolved task activity

In [ ]:
n_skip_BOLD = int(TRLength)
# Temporally downsample task timing to seconds (before convolving)
timingconv = tasktiming[:,::n_skip_BOLD]
timingconv.shape = (timingconv.shape[1],)
hrfsample_rate=1.0 # HRF is sampled at seconds
hrfsample_times = np.arange(0, 30, hrfsample_rate, dtype=float)
hrf_at_simsample = model.hrf(hrfsample_times)
hrfconvtime = np.convolve(timingconv, hrf_at_simsample)
n_to_remove = len(hrf_at_simsample) - 1
convolved = hrfconvtime[:-n_to_remove]
# Output
timing_convolved_downsampled = convolved

In [ ]:
task_betas_postfmri, task_resids_postfmri, task_tvals_postfmri, task_betas_ci_postfmri = model.runTaskGLM(task_bold, timing_convolved_downsampled)

Results of GLM. Vertical lines indicate network boundaries. Stimulated nodes are the first eight in the hub network.

In [ ]:
plt.plot(task_tvals_postfmri)
plt.ylabel('Uncorrected t-value',fontsize=14)
plt.xlabel('Node',fontsize=14)
plt.axhline(y=3,linewidth=2, color='red', ls = "--")
plt.axvline(x=35,linewidth=2, color='black', ls = "--")
plt.axvline(x=70,linewidth=2, color='black', ls = "--")

In [ ]:
uperror = task_betas_ci_postfmri[:,1]
downerror = task_betas_ci_postfmri[:,0]
errorbars = np.concatenate(([uperror], [downerror]), axis=0)

plt.errorbar(np.arange(totalnodes), task_betas_postfmri, yerr=errorbars, fmt = 'o')
plt.ylabel('Beta (with 95% CI)',fontsize=14)
plt.xlabel('Node',fontsize=14)

In [ ]:
hub_stim_nodes = stim_nodes